# Notebook eng from Chap 8 Bulldozer Feature Engineering

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from pandas.api.types import is_string_dtype, is_object_dtype, is_categorical_dtype

from rfpimp import *  # feature importance plot

bookcolors = {
    'crimson': '#a50026', 'red': '#d73027',
    'redorange': '#f46d43', 'orange': '#fdae61',
    'yellow': '#fee090', 'sky': '#e0f3f8',
    'babyblue': '#abd9e9', 'lightblue': '#74add1',
    'blue': '#4575b4', 'purple': '#313695'}

def test(X, y, n_estimators=50):
    rf = RandomForestRegressor(n_estimators=n_estimators, n_jobs=-1, oob_score=True)
    rf.fit(X, y)
    oob = rf.oob_score_
    n = rfnnodes(rf)
    h = np.median(rfmaxdepths(rf))
    print(f"OOB R^2 {oob:.5f} using {n:,d} tree nodes with {h} median tree height")
    return rf, oob

def df_string_to_cat(df):
    for col in df.columns:
        if is_string_dtype(df[col]):
            df[col] = df[col].astype('category').cat.as_ordered()

def df_cat_to_catcode(df):
    for col in df.columns:
        if is_categorical_dtype(df[col]):
            df[col] = df[col].cat.codes + 1        

def fix_missing_num(df, colname):
    df[colname+'_na'] = pd.isnull(df[colname])
    df[colname].fillna(df[colname].median(), inplace=True)

In [ ]:
df_raw = pd.read_feather("data/bulldozer-train.feather")
df_raw = df_raw.iloc[-100_000:] # same 100,000 records as before
df = pd.read_feather("data/bulldozer-train-clean.feather")

In [ ]:
X, y = df.drop(['SalePrice','saledate'], axis=1), df['SalePrice']
rf, oob_clean = test(X, y, n_estimators=150)

In [ ]:
def df_split_dates(df,colname):
    df["saleyear"] = df[colname].dt.year
    df["salemonth"] = df[colname].dt.month
    df["saleday"] = df[colname].dt.day
    df["saledayofweek"] = df[colname].dt.dayofweek
    df["saledayofyear"] = df[colname].dt.dayofyear
    df[colname] = df[colname].astype(np.int64) # convert to seconds since 1970

In [ ]:
df_split_dates(df, 'saledate')

In [ ]:
X, y = df.drop('SalePrice', axis=1), df['SalePrice']
rf, oob_dates = test(X, y, n_estimators=150)

In [ ]:
df['age'] = df['saleyear'] - df['YearMade']

X, y = df.drop('SalePrice', axis=1), df['SalePrice']
rf, oob_age = test(X, y, n_estimators=150)

In [ ]:
I = importances(rf, X, y)
plot_importances(I.head(15))

In [ ]:
features = list(df.drop('SalePrice',axis=1).columns)
datefeatures = list(df.filter(regex=("sale*")).columns)
for f in datefeatures:
    features.remove(f)
features.remove('YearMade')
features.remove('age')
features += [['YearMade','age']+datefeatures]
I = importances(rf, X, y, features=features)
plot_importances(I.head(15))

In [ ]:
fig,ax = plt.subplots()
df_small = df.sample(n=5_000) # don't draw too many dots
ax.scatter(df_small['age'], df_small['SalePrice'],
           alpha=0.03, c=bookcolors['blue'])
ax.set_ylabel("SalePrice")
ax.set_xlabel("Age in years")

In [ ]:
temp = df_raw.fillna('nan') # original dataset
temp = temp.groupby('ProductSize').mean()
temp[['SalePrice']].sort_values('SalePrice').plot.barh()

In [ ]:
sizes = {None:0, 'Mini':1, 'Compact':1, 'Small':2, 'Medium':3,
         'Large / Medium':4, 'Large':5}
df['ProductSize'] = df_raw['ProductSize'].map(sizes).values

In [ ]:
X, y = df.drop('SalePrice', axis=1), df['SalePrice']
rf, oob_ProductSize = test(X, y, n_estimators=150)

In [ ]:
df['Hydraulics_Flow'] = df_raw['Hydraulics_Flow'].values
df['Hydraulics_Flow'] = df['Hydraulics_Flow'].replace('None or Unspecified', np.nan)
onehot = pd.get_dummies(df['Hydraulics_Flow'],
                        prefix='Hydraulics_Flow',
                        dtype=bool)

In [ ]:
del df['Hydraulics_Flow']
df = pd.concat([df, onehot], axis=1)

In [ ]:
X, y = df.drop('SalePrice', axis=1), df['SalePrice']
rf, oob_Hydraulics_Flow = test(X, y, n_estimators=150)

In [ ]:
I = importances(rf, X, y)
plot_importances(I.head(20))

In [ ]:
df['Enclosure'] = df_raw['Enclosure'].values
df['Enclosure'] = df['Enclosure'].replace('EROPS w AC', 'EROPS AC')
df['Enclosure'] = df['Enclosure'].replace('None or Unspecified', np.nan)
df['Enclosure'] = df['Enclosure'].replace('NO ROPS', np.nan)

In [ ]:
temp = df.groupby('Enclosure').mean()
temp[['SalePrice']].sort_values('SalePrice').plot.barh()

In [ ]:
onehot = pd.get_dummies(df['Enclosure'],
                        prefix='Enclosure',
                        dtype=bool)
del df['Enclosure']
df = pd.concat([df, onehot], axis=1)
X, y = df.drop('SalePrice', axis=1), df['SalePrice']
rf, oob_Enclosure = test(X, y, n_estimators=150)

In [ ]:
I = importances(rf, X, y)
plot_importances(I.head(20))

In [ ]:
temp = df_raw.groupby('fiProductClassDesc').mean()
temp[['SalePrice']].sort_values('SalePrice').head(15).plot.barh()

In [ ]:
# careful when copying between dataframes; use .values
df_split = df_raw.fiProductClassDesc.str.split(' - ',expand=True).values
df['fiProductClassDesc'] = df_split[:,0] 
df['fiProductClassSpec'] = df_split[:,1] # temporary column

In [ ]:
pattern = r'([0-9.\+]*)(?: to ([0-9.\+]*)|\+) ([a-zA-Z ]*)'
df_split = df['fiProductClassSpec'].str.extract(pattern, expand=True).values
df['fiProductClassSpec_lower'] = pd.to_numeric(df_split[:,0])
df['fiProductClassSpec_upper'] = pd.to_numeric(df_split[:,1])
df['fiProductClassSpec_units'] = df_split[:,2]
del df['fiProductClassSpec'] # remove temporary column

In [ ]:
fix_missing_num(df, 'fiProductClassSpec_lower')
fix_missing_num(df, 'fiProductClassSpec_upper')
# label encode fiProductClassDesc fiProductClassSpec_units
df_string_to_cat(df)
df_cat_to_catcode(df)

In [ ]:
X, y = df.drop('SalePrice', axis=1), df['SalePrice']
rf, oob_fiProductClassDesc = test(X, y, n_estimators=150)

In [ ]:
X, y = df.drop('SalePrice', axis=1), df['SalePrice']
y = np.log(y)
rf, oob_log = test(X, y, n_estimators=150)

In [ ]:
I = importances(rf, X, y)
plot_importances(I.head(30))

In [ ]:
features = ['Baseline', 'dates', 'age', 'ProductSize', 
            'Hydraulics_Flow', 'Enclosure', 
            'fiProductClassDesc', 'log']
scores   = [oob_clean, oob_dates, oob_age, oob_ProductSize, 
            oob_Hydraulics_Flow, oob_Enclosure, 
            oob_fiProductClassDesc, oob_log]

fig,ax = plt.subplots(figsize=(6,4))
ax.plot(features, scores, 'o-', c=bookcolors['blue'])
plt.xticks(rotation=60)
ax.set_ylabel("OOB R^2 Score")
ax.set_xlabel("Change made to model")